**Jasmin Putri Jelita - 202110370311075**

**Tazkia Damayanti - 202110370311090**

**UTS TEMU KEMBALI INFORMASI**

In [13]:
import pandas as pd
import numpy as np
import os
import nltk
import re
import operator
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tazkiiia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tazkiiia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_excel('D:\College\Semester 7\Temu Kembali Informasi (TKI)\Dataset-Narkotika_075_090\Overview\Overview.xlsx')
df.head(5)

,No Putusan,Lembaga Peradilan,Barang Bukti,Amar Putusan
0,249/Pid.Sus/2024/PN Mtr,PN MATARAM,: - 1 (satu) bungkus Narkotika jenis Shabu dib...,M E N G A D I L I: Menyatakan Terdakwa Burhanu...
1,251/Pid.Sus/2024/PN Mtr,PN MATARAM,: 22 (dua puluh dua) bungkus plastik bening ya...,MENGADILI: Menyatakan Terdakwa Mustar Irawan A...
2,261/Pid.Sus/2024/PN Mtr,PN MATARAM,: 1 (satu) buah plastic klip bening berisi kri...,M E N G A D I L I: Menyatakan Terdakwa Syarifu...
3,268/Pid.Sus/2024/PN Mtr,PN MATARAM,: 1. (satu) buah wadah berbentuk gelas warna h...,M E N G A D I L I: Menyatakan Terdakwa terdakw...
4,299/Pid.Sus/2024/PN Mtr,PN MATARAM,: - 1 (satu) dompet warna hitam - 1 (satu) uni...,Menyatakan Terdakwa Sri Hikmawati Binti H. Bah...


# EDA (Exploratory Data Analysis)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   No Putusan         50 non-null     object
 1   Lembaga Peradilan  50 non-null     object
 2   Barang Bukti       50 non-null     object
 3   Amar Putusan       50 non-null     object
dtypes: object(4)
memory usage: 1.7+ KB


In [4]:
df.isnull().sum()

No Putusan           0
Lembaga Peradilan    0
Barang Bukti         0
Amar Putusan         0
dtype: int64

# Pre-processing

In [8]:
def data_cleaning(text):
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace('_', '')
    text = text.replace('m e n g a d i l i', 'mengadili')
    text = text.replace('M E N G A D I L I', 'mengadili')
    return text.lower()

df['Barang Bukti'] = df['Barang Bukti'].apply(data_cleaning)
df['Amar Putusan'] = df['Amar Putusan'].apply(data_cleaning)
df.head()

,No Putusan,Lembaga Peradilan,Barang Bukti,Amar Putusan,data_baru
0,249/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu bungkus narkotika jenis shabu dibungk...,mengadili menyatakan terdakwa burhanudin alias...,1 satu bungkus narkotika jenis shabu dibungk...
1,251/Pid.Sus/2024/PN Mtr,PN MATARAM,22 dua puluh dua bungkus plastik bening yang ...,mengadili menyatakan terdakwa mustar irawan al...,22 dua puluh dua bungkus plastik bening yang ...
2,261/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu buah plastic klip bening berisi krista...,mengadili menyatakan terdakwa syarifudin telah...,1 satu buah plastic klip bening berisi krista...
3,268/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu buah wadah berbentuk gelas warna hitam...,mengadili menyatakan terdakwa terdakwa i muham...,1 satu buah wadah berbentuk gelas warna hitam...
4,299/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu dompet warna hitam 1 satu unit atm b...,menyatakan terdakwa sri hikmawati binti h bahr...,1 satu dompet warna hitam 1 satu unit atm b...


In [9]:
df['Merge Data'] = df['Barang Bukti'] + ' ' + df['Amar Putusan']
df.head()

,No Putusan,Lembaga Peradilan,Barang Bukti,Amar Putusan,data_baru,Merge Data
0,249/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu bungkus narkotika jenis shabu dibungk...,mengadili menyatakan terdakwa burhanudin alias...,1 satu bungkus narkotika jenis shabu dibungk...,1 satu bungkus narkotika jenis shabu dibungk...
1,251/Pid.Sus/2024/PN Mtr,PN MATARAM,22 dua puluh dua bungkus plastik bening yang ...,mengadili menyatakan terdakwa mustar irawan al...,22 dua puluh dua bungkus plastik bening yang ...,22 dua puluh dua bungkus plastik bening yang ...
2,261/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu buah plastic klip bening berisi krista...,mengadili menyatakan terdakwa syarifudin telah...,1 satu buah plastic klip bening berisi krista...,1 satu buah plastic klip bening berisi krista...
3,268/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu buah wadah berbentuk gelas warna hitam...,mengadili menyatakan terdakwa terdakwa i muham...,1 satu buah wadah berbentuk gelas warna hitam...,1 satu buah wadah berbentuk gelas warna hitam...
4,299/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu dompet warna hitam 1 satu unit atm b...,menyatakan terdakwa sri hikmawati binti h bahr...,1 satu dompet warna hitam 1 satu unit atm b...,1 satu dompet warna hitam 1 satu unit atm b...


In [10]:
def tokenization(text, lang='indonesian'):
    if not isinstance(text, str):
        text = str(text)
    stop_words = set(stopwords.words(lang))
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    return filtered_tokens

prepocessed_data = df['Merge Data'].apply(tokenization)
df['Prepocessed Data'] = prepocessed_data
df.head()

,No Putusan,Lembaga Peradilan,Barang Bukti,Amar Putusan,data_baru,Merge Data,Prepocessed Data
0,249/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu bungkus narkotika jenis shabu dibungk...,mengadili menyatakan terdakwa burhanudin alias...,1 satu bungkus narkotika jenis shabu dibungk...,1 satu bungkus narkotika jenis shabu dibungk...,"[1, bungkus, narkotika, jenis, shabu, dibungku..."
1,251/Pid.Sus/2024/PN Mtr,PN MATARAM,22 dua puluh dua bungkus plastik bening yang ...,mengadili menyatakan terdakwa mustar irawan al...,22 dua puluh dua bungkus plastik bening yang ...,22 dua puluh dua bungkus plastik bening yang ...,"[22, puluh, bungkus, plastik, bening, berisi, ..."
2,261/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu buah plastic klip bening berisi krista...,mengadili menyatakan terdakwa syarifudin telah...,1 satu buah plastic klip bening berisi krista...,1 satu buah plastic klip bening berisi krista...,"[1, buah, plastic, klip, bening, berisi, krist..."
3,268/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu buah wadah berbentuk gelas warna hitam...,mengadili menyatakan terdakwa terdakwa i muham...,1 satu buah wadah berbentuk gelas warna hitam...,1 satu buah wadah berbentuk gelas warna hitam...,"[1, buah, wadah, berbentuk, gelas, warna, hita..."
4,299/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu dompet warna hitam 1 satu unit atm b...,menyatakan terdakwa sri hikmawati binti h bahr...,1 satu dompet warna hitam 1 satu unit atm b...,1 satu dompet warna hitam 1 satu unit atm b...,"[1, dompet, warna, hitam, 1, unit, atm, bca, n..."


In [11]:
df['Processed Text'] = df['Prepocessed Data'].apply(lambda tokens: ' '.join(tokens))
df.head()

,No Putusan,Lembaga Peradilan,Barang Bukti,Amar Putusan,data_baru,Merge Data,Prepocessed Data,Processed Text
0,249/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu bungkus narkotika jenis shabu dibungk...,mengadili menyatakan terdakwa burhanudin alias...,1 satu bungkus narkotika jenis shabu dibungk...,1 satu bungkus narkotika jenis shabu dibungk...,"[1, bungkus, narkotika, jenis, shabu, dibungku...",1 bungkus narkotika jenis shabu dibungkus plas...
1,251/Pid.Sus/2024/PN Mtr,PN MATARAM,22 dua puluh dua bungkus plastik bening yang ...,mengadili menyatakan terdakwa mustar irawan al...,22 dua puluh dua bungkus plastik bening yang ...,22 dua puluh dua bungkus plastik bening yang ...,"[22, puluh, bungkus, plastik, bening, berisi, ...",22 puluh bungkus plastik bening berisi narkoti...
2,261/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu buah plastic klip bening berisi krista...,mengadili menyatakan terdakwa syarifudin telah...,1 satu buah plastic klip bening berisi krista...,1 satu buah plastic klip bening berisi krista...,"[1, buah, plastic, klip, bening, berisi, krist...",1 buah plastic klip bening berisi kristal beni...
3,268/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu buah wadah berbentuk gelas warna hitam...,mengadili menyatakan terdakwa terdakwa i muham...,1 satu buah wadah berbentuk gelas warna hitam...,1 satu buah wadah berbentuk gelas warna hitam...,"[1, buah, wadah, berbentuk, gelas, warna, hita...",1 buah wadah berbentuk gelas warna hitam didal...
4,299/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu dompet warna hitam 1 satu unit atm b...,menyatakan terdakwa sri hikmawati binti h bahr...,1 satu dompet warna hitam 1 satu unit atm b...,1 satu dompet warna hitam 1 satu unit atm b...,"[1, dompet, warna, hitam, 1, unit, atm, bca, n...",1 dompet warna hitam 1 unit atm bca nomor 6019...


# Indexing

In [14]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Processed Text'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df.head()


,0005,0007,0036,0037,0038,0041,0042,0043,0045,0050,...,xl,yamaha,yani,yayat,ym,yona,yuda,yume,yusuf,zan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.230788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.135672,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
def create_inverted_index(df, column_name):
    inverted_index = defaultdict(set)
    for index, tokens in df[column_name].items():
        for token in tokens:
            inverted_index[token.lower()].add(index)
    return inverted_index

inverted_index = create_inverted_index(df, 'Prepocessed Data')

def print_inverted_index(index, limit=10):
    print("Inverted Index (showing up to {} entries per word):".format(limit))
    for word, doc_ids in index.items():
        print(f"{word}: {sorted(list(doc_ids))[:limit]}")

print_inverted_index(inverted_index)


Inverted Index (showing up to 10 entries per word):
1: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10]
bungkus: [0, 1, 5, 6, 7, 9, 17, 18, 19, 20]
narkotika: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
jenis: [0, 1, 2, 3, 5, 6, 7, 8, 9, 10]
shabu: [0, 6, 7, 17, 18, 19, 20, 22, 25, 28]
dibungkus: [0, 7, 17, 18, 20, 30, 32, 40, 46, 47]
plastik: [0, 1, 3, 5, 6, 7, 8, 9, 10, 17]
klip: [0, 2, 3, 5, 6, 7, 8, 10, 13, 16]
dililit: [0, 20, 40]
isolsi: [0]
warna: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10]
kuning: [0, 9, 23, 29, 35, 43]
unit: [0, 2, 4, 6, 7, 9, 11, 12, 18, 19]
heandphone: [0]
merek: [0, 7, 18, 35, 43]
samsung: [0, 3, 6, 18, 24, 26, 30, 31, 32, 40]
hitam: [0, 3, 4, 6, 7, 9, 10, 11, 12, 13]
pelindung: [0]
biru: [0, 6, 7, 8, 9, 11, 17, 18, 20, 23]
berisi: [0, 1, 2, 3, 5, 6, 8, 9, 10, 13]
sim: [0, 4, 6, 7, 17, 18, 22, 28, 35, 40]
card: [0, 4, 6, 7, 17, 22, 28, 35, 43, 46]
xl: [0, 6, 20, 22, 30, 31, 32, 35, 41, 43]
087812203428: [0]
readmi: [0, 30, 32]
087864354545: [0]
nokia: [0, 6, 25, 28, 35, 36, 39, 43, 44, 46]
muda: [0

In [17]:
def create_inverted_index(df, column_name):
    inverted_index = defaultdict(set)
    for index, tokens in df[column_name].items():
        for token in tokens:
            inverted_index[token.lower()].add(index)
    return inverted_index

inverted_index = create_inverted_index(df, 'Prepocessed Data')

def print_inverted_index(index):
    data = {word: sorted(list(doc_ids)) for word, doc_ids in index.items()}
    df_index = pd.DataFrame(list(data.items()), columns=['Word', 'Document IDs'])
    print(df_index)

print_inverted_index(inverted_index)


                     Word                                       Document IDs
0                       1  [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
1                 bungkus  [0, 1, 5, 6, 7, 9, 17, 18, 19, 20, 21, 22, 23,...
2               narkotika  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
3                   jenis  [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 13, 14, 16, 17...
4                   shabu  [0, 6, 7, 17, 18, 19, 20, 22, 25, 28, 29, 30, ...
...                   ...                                                ...
1377              selama5                                               [49]
1378       penangkapandan                                               [49]
1379  terdakwadikurangkan                                               [49]
1380        terdakwatetap                                               [49]
1381     menetapkanbarang                                               [49]

[1382 rows x 2 columns]


# Modeling

In [18]:
X = df['Merge Data']
y = df['Amar Putusan']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [20]:
nb_classifier = MultinomialNB()
rf_classifier = RandomForestClassifier()
clf = VotingClassifier(estimators=[('nb', nb_classifier), ('rf', rf_classifier)], voting='soft')
clf.fit(X_train_vectorized,y_train)

VotingClassifier(estimators=[('nb', MultinomialNB()),
                             ('rf', RandomForestClassifier())],
                 voting='soft')

In [21]:
def search(query):
    query_cleaned = data_cleaning(query)
    query_tokens = word_tokenize(query_cleaned)

    query_vectorized = vectorizer.transform([' '.join(query_tokens)])

    prediction = clf.predict(query_vectorized)
    result = df[df['Amar Putusan'] == prediction[0]]
    return result

In [25]:
query = 'M E N G A D I L I: Menyatakan Terdakwa Syarifudin telah terbukti secara sah '
result = search(query)

print("Hasil pencarian untuk query '{}':".format(query))
res = pd.DataFrame(result)
res[['No Putusan', 'Lembaga Peradilan', 'Barang Bukti','Amar Putusan']]

Hasil pencarian untuk query 'M E N G A D I L I: Menyatakan Terdakwa Syarifudin telah terbukti secara sah ':


,No Putusan,Lembaga Peradilan,Barang Bukti,Amar Putusan
2,261/Pid.Sus/2024/PN Mtr,PN MATARAM,1 satu buah plastic klip bening berisi krista...,mengadili menyatakan terdakwa syarifudin telah...
